In [ ]:
# -*- coding: utf-8 -*-
"""
# TCC Nourish: Simulação de Onboarding e Estrutura de Recomendação
[cite_start]Este notebook contém a implementação do fluxo de onboarding do usuário e a estruturação inicial da base de dados, conforme o projeto do TCC "Nourish"[cite: 13, 16].

A seção final (Módulo 3) apresenta a arquitetura conceitual para a transição do sistema de regras (MVP) para um Sistema de Recomendação baseado em Machine Learning.
"""

# --- MÓDULO 1: Estrutura de Dados e Base de Dados Simulada ---

import time
import re
import pandas as pd # Importação para fins de ML na seção 3

## Base de Dados Simulado (do PDF)

RECEITAS = {
    "cafe_da_manha": [
        {
            "nome": "Frutas com mel",
            "ingredientes": [
                "4 fatias de abacaxi picadas",
                "1 pera com casca cortada em cubos",
                "1 maçã verde com casca cortada em cubos",
                "1 maçã vermelha com casca cortada em cubos",
                "meia xícara (chá) de mel",
                "6 bolas de sorvete KIBON sabor creme" # Nota: Alto açúcar/carboidrato
            ],
            "modo_preparo": [
                "Preaqueça o forno em temperatura média (180º C).",
                "Divida o abacaxi picado em 6 partes e coloque cada uma no centro de um quadrado de papel alumínio. Repita a operação com a pera e as maçãs.",
                "Espalhe o mel sobre as frutas e feche o papel alumínio apertando as extremidades, formando um pacote.",
                "Coloque os em uma assadeira média (33 x 23 cm de diâmetro) e leve ao forno por 25 minutos.",
                "Coloque cada pacote em um prato de sobremesa e abra com cuidado, mantendo as laterais do papel levantadas.",
                "Coloque uma bola de sorvete KIBON sabor creme no centro do pacote e sirva em seguida."
            ],
            "nutricional": {
                "calorias": 158, # kcal
                "carboidratos": 25, # g
                "proteinas": 59, # g (Valor do PDF parece muito alto para frutas)
                "gordura": 15, # g
                "acucar": 20 # g
            }
        }
    ],
    "almoco_jantar": [
        {
            "nome": "Nhoque de batata doce",
            "dificuldade": "FÁCIL",
            "tempo_preparo_min": 25, # MIN
            "ingredientes": [
                "2 batatas doces pequenas",
                "1 colher de sopa rasa de sal (para massa)",
                "Aproximadamente 1 copo de farinha de trigo",
                "3 dentes de alho picados",
                "1 colher de sopa de azeite",
                "2 copos de molho de tomate",
                "1 colher de chá de sal (para molho)"
            ],
            "modo_preparo": [
                "Em uma panela, adicione a água (o suficiente para cozinhar a batata) e deixe ferver.",
                "Descasque e corte as batatas em cubinhos.",
                "Coloque elas para cozinhar até ficarem macias.",
                "Escorra a água e amasse elas com um garfo até formar um purê.",
                "Acrescente o sal, a farinha (peneirada) e misture até formar uma massa.",
                "Polvilhe farinha na bancada para não grudar.",
                "Faça rolinhos com a massa, corte os pedaços de nhoque do tamanho que desejar e coloque eles para cozinhar em água quente.",
                "Assim que eles começarem a boiar, retire e coloque-os em um refratário.",
                "Para o molho, doure o alho em uma panela com azeite.",
                "Adicione o molho de tomate, o sal e deixe cozinhar por alguns minutos.",
                "Despeje o molho no refratário com o nhoque, misture e sirva. Bom apetite."
            ]
        }
    ]
}

## Classe para armazenar os dados do Usuário

class Usuario:
    """ Armazena os dados do perfil do usuário, baseados nas telas de onboarding do PDF. """
    def __init__(self):
        self.genero = None # P. 8
        self.altura_m = 0.0 # P. 9
        self.peso_kg = 0.0 # P. 10
        self.faz_exercicio = None # P. 11
        self.objetivo = None # P. 12
        self.motivacao = None # P. 13
        self.preferencias = [] # P. 57
        self.imc = 0.0

    def calcular_imc(self):
        """ Calcula o Índice de Massa Corporal (IMC). """
        if self.altura_m > 0:
            self.imc = self.peso_kg / (self.altura_m ** 2)
            return self.imc
        return 0.0

    def __str__(self):
        """ Retorna um resumo formatado do perfil do usuário. """
        exercicio_str = "Sim" if self.faz_exercicio else "Não"
        self.calcular_imc()

        return f"""
-----------------------------------
Perfil de Usuário 'Nourish' Criado
-----------------------------------
Gênero: {self.genero}
Altura: {self.altura_m} m
Peso: {self.peso_kg} kg
IMC: {self.imc:.2f}
Faz Exercício: {exercicio_str}
Objetivo Principal: {self.objetivo}
Motivação: {self.motivacao}
Preferências: {', '.join(self.preferencias)}
-----------------------------------
"""


# --- MÓDULO 2: Funções de Onboarding e Validação (Simulação da Interface) ---

def validar_input_lista(prompt: str, opcoes_validas: list) -> str:
    """ Força o usuário a escolher uma opção de uma lista. """
    print(f"\n--- {prompt} ---")
    for i, opcao in enumerate(opcoes_validas, 1):
        print(f"  {i}. {opcao}")

    while True:
        escolha = input("Digite o número da sua escolha: ")
        if escolha.isdigit() and 1 <= int(escolha) <= len(opcoes_validas):
            return opcoes_validas[int(escolha) - 1]
        else:
            print(f"Escolha inválida. Por favor, digite um número entre 1 e {len(opcoes_validas)}.")

def validar_input_float(prompt: str, min_val: float, max_val: float) -> float:
    """ Força o usuário a inserir um número (float) num intervalo. """
    print(f"\n--- {prompt} ---")
    while True:
        entrada = input("Digite o valor: ").replace(',', '.')

        if not re.match(r"^\d+(\.\d+)?$", entrada):
            print("Entrada inválida. Use apenas números (ex: 70.5 ou 1.75).")
            continue

        try:
            valor = float(entrada)
            if min_val <= valor <= max_val:
                return valor
            else:
                print(f"Valor fora do intervalo. Insira algo entre {min_val} e {max_val}.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")

def validar_input_multiplo(prompt: str, opcoes_validas: list) -> list:
    """ Permite ao usuário escolher múltiplas opções de uma lista. """
    print(f"\n--- {prompt} ---")
    print("(Digite os números separados por vírgula, ex: 1, 3, 5)")
    print("  0. Nenhuma das opções / Não tenho preferências")
    for i, opcao in enumerate(opcoes_validas, 1):
        print(f"  {i}. {opcao}")

    escolhas_finais = []
    while True:
        try:
            entrada = input("Digite os números das suas escolhas: ")
            numeros_str = entrada.split(',')

            numeros_int = []
            for num in numeros_str:
                num_limpo = num.strip()
                if not num_limpo.isdigit():
                    raise ValueError(f"'{num_limpo}' não é um número válido.")
                numeros_int.append(int(num_limpo))

            if 0 in numeros_int:
                return ["Não tenho preferências"]

            escolhas_finais = []
            for n in numeros_int:
                if 1 <= n <= len(opcoes_validas):
                    escolhas_finais.append(opcoes_validas[n - 1])
                else:
                    raise ValueError(f"Número {n} está fora do intervalo (1-{len(opcoes_validas)}).")

            if not escolhas_finais:
                print("Você não selecionou nenhuma opção válida.")
                continue

            return list(set(escolhas_finais)) # Remove duplicatas

        except ValueError as e:
            print(f"Erro: {e}. Por favor, tente novamente (ex: 1, 3, 5).")

def coletar_dados_usuario(usuario: Usuario):
    """Executa o fluxo de perguntas de onboarding do app Nourish."""

    print("==========================================")
    print("       Bem-vindo ao Nourish!")
    print("==========================================")
    time.sleep(1)

    # 1. Gênero
    generos = ["Feminino", "Masculino", "Outros"]
    usuario.genero = validar_input_lista("Qual seu gênero?", generos)

    # 2. Altura (em metros)
    usuario.altura_m = validar_input_float("Qual sua altura? (m)", 0.5, 3.0)

    # 3. Peso (em kg)
    usuario.peso_kg = validar_input_float("Qual seu peso atual? (kg)", 10.0, 300.0)

    # 4. Prática de Exercício
    exercicios = ["Faço", "Não faço"]
    resposta_exercicio = validar_input_lista("Você faz exercício físico atualmente?", exercicios)
    usuario.faz_exercicio = (resposta_exercicio == "Faço")

    # 5. Objetivo Principal
    objetivos = ["Perder peso", "Ganhar massa", "Manter peso", "Reeducação alimentar"]
    usuario.objetivo = validar_input_lista("Qual seu objetivo?", objetivos)

    # 6. Motivação
    motivacoes = ["Melhorar qualidade de vida", "Problema de saúde", "Controlar minha dieta", "Não desejo compartilhar"]
    usuario.motivacao = validar_input_lista("Nos conte mais o por quê você quer chegar nesse objetivo:", motivacoes)

    # 7. Preferências Alimentares
    preferencias_opcoes = ["Carnes vermelhas", "Frango", "Massas", "Legumes", "Frutas", "Ovo"]
    usuario.preferencias = validar_input_multiplo("O que você gosta de comer?", preferencias_opcoes)

    # Cálculo final do IMC e exibição do resumo
    usuario.calcular_imc()
    print(usuario)

    print("\n✅ Onboarding concluído com sucesso!")
    return usuario

# --- MÓDULO 3: Estrutura para Machine Learning (ML) ---

# O modelo de ML para recomendação exige que a base de dados de receitas seja estruturada
# para cálculo matricial (vetores de features).

def criar_dataframe_receitas(receitas_dict):
    """ Converte o dicionário de receitas em um DataFrame para análise de ML/Regras. """
    dados = []
    for categoria, lista_receitas in receitas_dict.items():
        for receita in lista_receitas:
            row = {
                'categoria': categoria,
                'nome': receita['nome'],
                'dificuldade': receita.get('dificuldade', 'N/A'),
                'tempo_preparo_min': receita.get('tempo_preparo_min', None),
                'calorias': receita['nutricional']['calorias'] if 'nutricional' in receita else None,
                'carboidratos': receita['nutricional']['carboidratos'] if 'nutricional' in receita else None,
                'proteinas': receita['nutricional']['proteinas'] if 'nutricional' in receita else None,
                'gordura': receita['nutricional']['gordura'] if 'nutricional' in receita else None,
                # Feature Engineering de Regras de Negócio
                'is_baixo_carb': 1 if receita['nome'] not in ['Frutas com mel', 'Nhoque de batata doce'] else 0, # Simulação de regra
                'is_alto_acucar': 1 if 'sorvete' in ''.join(receita['ingredientes']).lower() else 0
            }
            dados.append(row)
    return pd.DataFrame(dados)

def simular_recomendacao_regra(usuario: Usuario, df_receitas: pd.DataFrame):
    """ Função de Recomendação baseada em Regras de Negócio (MVP). """
    print("\n--- Simulação de Recomendação (MVP: Regras) ---")

    if usuario.objetivo == "Perder peso" and "Baixo Carboidrato" in usuario.preferencias:
        print(f"Filtro Ativo: {usuario.objetivo} + Baixo Carboidrato")

        # Filtrar receitas que não são alto açúcar e são consideradas baixo carboidrato
        receitas_filtradas = df_receitas[
            (df_receitas['is_alto_acucar'] == 0) &
            (df_receitas['is_baixo_carb'] == 1)
        ]

        if not receitas_filtradas.empty:
            sugestao = receitas_filtradas.sample(1)
            print(f"\n✅ Sugestão Otimizada: {sugestao['nome'].iloc[0]} ({sugestao['categoria'].iloc[0]})")
        else:
            # Note que as receitas do PDF são inadequadas para esta regra
            print("\n❌ Nenhuma receita da base de dados atual (PDF) atende ao critério 'Perda de Peso' + 'Baixo Carboidrato'.")
            print("Seria necessária uma receita como 'Omelete com Espinafre'.")

    else:
        # Recomendação padrão: receita mais simples/rápida
        print("\nSugestão Padrão: Nhoque de batata doce (Almoço/Jantar)")

    print("--------------------------------------------------")


# --- EXECUÇÃO DO NOTEBOOK ---

# 1. Coletar dados do usuário (simulação interativa)
usuario_novo = Usuario()
usuario_completo = coletar_dados_usuario(usuario_novo)

# 2. Estruturar o Dataset de Receitas
df_receitas = criar_dataframe_receitas(RECEITAS)
# print("\nEstrutura do Dataset de Receitas para ML:")
# display(df_receitas) # Use 'display' em ambientes Jupyter/Colab

# 3. Executar a simulação de recomendação
simular_recomendacao_regra(usuario_completo, df_receitas)

       Bem-vindo ao Nourish!

--- Qual seu gênero? ---
  1. Feminino
  2. Masculino
  3. Outros


In [2]:
def coletar_dados_usuario(usuario: Usuario):
    """Coleta todas as informações necessárias para preencher o perfil do usuário."""

    print("\n--- Coleta de Dados Pessoais ---")

    # 1. Gênero
    generos = ["Masculino", "Feminino", "Outro/Prefiro não dizer"]
    usuario.genero = validar_input_lista("Selecione seu Gênero", generos)

    # 2. Altura (em metros)
    usuario.altura_m = validar_input_float("Informe sua Altura em metros (ex: 1.75)", 0.5, 3.0)

    # 3. Peso (em kg)
    usuario.peso_kg = validar_input_float("Informe seu Peso em quilogramas (ex: 75.5)", 10.0, 300.0)

    # 4. Prática de Exercício
    exercicios = ["Sim, pratico regularmente", "Não pratico regularmente"]
    resposta_exercicio = validar_input_lista("Você pratica exercícios físicos regularmente?", exercicios)
    usuario.faz_exercicio = (resposta_exercicio == "Sim, pratico regularmente")

    # 5. Objetivo Principal
    objetivos = ["Perda de Peso", "Ganho de Massa Muscular", "Manutenção/Saúde Geral"]
    usuario.objetivo = validar_input_lista("Qual é o seu principal objetivo com a alimentação?", objetivos)

    # 6. Motivação
    motivacoes = ["Melhora da saúde", "Estética/Visual", "Performance esportiva", "Recomendação médica"]
    usuario.motivacao = validar_input_lista("Qual é a sua principal motivação?", motivacoes)

    # 7. Preferências Alimentares
    preferencias_opcoes = ["Vegetariano", "Vegano", "Sem Glúten", "Sem Lactose", "Baixo Carboidrato", "Alto Teor Proteico"]
    usuario.preferencias = validar_input_multiplo("Quais restrições ou preferências alimentares você possui?", preferencias_opcoes)

    # Cálculo final do IMC e exibição do resumo
    usuario.calcular_imc()
    print(usuario)

    print("\n✅ Onboarding concluído com sucesso!")
    return usuario

# --- Início da Execução ---
meu_usuario = Usuario()
meu_usuario = coletar_dados_usuario(meu_usuario)

NameError: name 'Usuario' is not defined

In [4]:
# -*- coding: utf-8 -*-
"""
# TCC Nourish: Simulação de Onboarding e Estrutura de Recomendação
[cite_start]Este notebook contém a implementação do fluxo de onboarding do usuário e a estruturação inicial da base de dados, conforme o projeto do TCC "Nourish"[cite: 13, 16].

A seção final (Módulo 3) apresenta a arquitetura conceitual para a transição do sistema de regras (MVP) para um Sistema de Recomendação baseado em Machine Learning.
"""

# --- MÓDULO 1: Estrutura de Dados e Base de Dados Simulada ---

import time
import re
import pandas as pd # Importação para fins de ML na seção 3

## Base de Dados Simulado (do PDF)

RECEITAS = {
    "cafe_da_manha": [
        {
            "nome": "Frutas com mel",
            "ingredientes": [
                "4 fatias de abacaxi picadas",
                "1 pera com casca cortada em cubos",
                "1 maçã verde com casca cortada em cubos",
                "1 maçã vermelha com casca cortada em cubos",
                "meia xícara (chá) de mel",
                "6 bolas de sorvete KIBON sabor creme" # Nota: Alto açúcar/carboidrato
            ],
            "modo_preparo": [
                "Preaqueça o forno em temperatura média (180º C).",
                "Divida o abacaxi picado em 6 partes e coloque cada uma no centro de um quadrado de papel alumínio. Repita a operação com a pera e as maçãs.",
                "Espalhe o mel sobre as frutas e feche o papel alumínio apertando as extremidades, formando um pacote.",
                "Coloque os em uma assadeira média (33 x 23 cm de diâmetro) e leve ao forno por 25 minutos.",
                "Coloque cada pacote em um prato de sobremesa e abra com cuidado, mantendo as laterais do papel levantadas.",
                "Coloque uma bola de sorvete KIBON sabor creme no centro do pacote e sirva em seguida."
            ],
            "nutricional": {
                [cite_start]"calorias": 158, # kcal [cite: 38]
                [cite_start]"carboidratos": 25, # g [cite: 38]
                [cite_start]"proteinas": 59, # g (Valor do PDF parece muito alto para frutas) [cite: 38]
                [cite_start]"gordura": 15, # g [cite: 38]
                [cite_start]"acucar": 20 # g [cite: 38]
            }
        }
    ],
    "almoco_jantar": [
        {
            "nome": "Nhoque de batata doce",
            "dificuldade": "FÁCIL",
            [cite_start]"tempo_preparo_min": 25, # MIN [cite: 48]
            "ingredientes": [
                "2 batatas doces pequenas",
                "1 colher de sopa rasa de sal (para massa)",
                "Aproximadamente 1 copo de farinha de trigo",
                "3 dentes de alho picados",
                "1 colher de sopa de azeite",
                "2 copos de molho de tomate",
                "1 colher de chá de sal (para molho)"
            ],
            "modo_preparo": [
                "Em uma panela, adicione a água (o suficiente para cozinhar a batata) e deixe ferver.",
                "Descasque e corte as batatas em cubinhos.",
                "Coloque elas para cozinhar até ficarem macias.",
                "Escorra a água e amasse elas com um garfo até formar um purê.",
                "Acrescente o sal, a farinha (peneirada) e misture até formar uma massa.",
                "Polvilhe farinha na bancada para não grudar.",
                "Faça rolinhos com a massa, corte os pedaços de nhoque do tamanho que desejar e coloque eles para cozinhar em água quente.",
                "Assim que eles começarem a boiar, retire e coloque-os em um refratário.",
                "Para o molho, doure o alho em uma panela com azeite.",
                "Adicione o molho de tomate, o sal e deixe cozinhar por alguns minutos.",
                "Despeje o molho no refratário com o nhoque, misture e sirva. Bom apetite."
            ]
        }
    ]
}

## Classe para armazenar os dados do Usuário

class Usuario:
    [cite_start]""" Armazena os dados do perfil do usuário, baseados nas telas de onboarding do PDF[cite: 16]. """
    def __init__(self):
        [cite_start]self.genero = None # P. 8 [cite: 20]
        [cite_start]self.altura_m = 0.0 # P. 9 [cite: 26]
        [cite_start]self.peso_kg = 0.0 # P. 10 [cite: 30]
        [cite_start]self.faz_exercicio = None # P. 11 [cite: 34]
        [cite_start]self.objetivo = None # P. 12 [cite: 35]
        [cite_start]self.motivacao = None # P. 13 [cite: 36]
        [cite_start]self.preferencias = [] # P. 57 [cite: 57]
        self.imc = 0.0

    def calcular_imc(self):
        """ Calcula o Índice de Massa Corporal (IMC). """
        if self.altura_m > 0:
            self.imc = self.peso_kg / (self.altura_m ** 2)
            return self.imc
        return 0.0

    def __str__(self):
        """ Retorna um resumo formatado do perfil do usuário. """
        exercicio_str = "Sim" if self.faz_exercicio else "Não"
        self.calcular_imc()

        return f"""
-----------------------------------
Perfil de Usuário 'Nourish' Criado
-----------------------------------
Gênero: {self.genero}
Altura: {self.altura_m} m
Peso: {self.peso_kg} kg
IMC: {self.imc:.2f}
Faz Exercício: {exercicio_str}
Objetivo Principal: {self.objetivo}
Motivação: {self.motivacao}
Preferências: {', '.join(self.preferencias)}
-----------------------------------
"""


# --- MÓDULO 2: Funções de Onboarding e Validação (Simulação da Interface) ---

def validar_input_lista(prompt: str, opcoes_validas: list) -> str:
    """ Força o usuário a escolher uma opção de uma lista. """
    print(f"\n--- {prompt} ---")
    for i, opcao in enumerate(opcoes_validas, 1):
        print(f"  {i}. {opcao}")

    while True:
        escolha = input("Digite o número da sua escolha: ")
        if escolha.isdigit() and 1 <= int(escolha) <= len(opcoes_validas):
            return opcoes_validas[int(escolha) - 1]
        else:
            print(f"Escolha inválida. Por favor, digite um número entre 1 e {len(opcoes_validas)}.")

def validar_input_float(prompt: str, min_val: float, max_val: float) -> float:
    """ Força o usuário a inserir um número (float) num intervalo. """
    print(f"\n--- {prompt} ---")
    while True:
        entrada = input("Digite o valor: ").replace(',', '.')

        if not re.match(r"^\d+(\.\d+)?$", entrada):
            print("Entrada inválida. Use apenas números (ex: 70.5 ou 1.75).")
            continue

        try:
            valor = float(entrada)
            if min_val <= valor <= max_val:
                return valor
            else:
                print(f"Valor fora do intervalo. Insira algo entre {min_val} e {max_val}.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")

def validar_input_multiplo(prompt: str, opcoes_validas: list) -> list:
    [cite_start]""" Permite ao usuário escolher múltiplas opções de uma lista[cite: 57]. """
    print(f"\n--- {prompt} ---")
    print("(Digite os números separados por vírgula, ex: 1, 3, 5)")
    print("  0. Nenhuma das opções / Não tenho preferências")
    for i, opcao in enumerate(opcoes_validas, 1):
        print(f"  {i}. {opcao}")

    escolhas_finais = []
    while True:
        try:
            entrada = input("Digite os números das suas escolhas: ")
            numeros_str = entrada.split(',')

            numeros_int = []
            for num in numeros_str:
                num_limpo = num.strip()
                if not num_limpo.isdigit():
                    raise ValueError(f"'{num_limpo}' não é um número válido.")
                numeros_int.append(int(num_limpo))

            if 0 in numeros_int:
                return ["Não tenho preferências"]

            escolhas_finais = []
            for n in numeros_int:
                if 1 <= n <= len(opcoes_validas):
                    escolhas_finais.append(opcoes_validas[n - 1])
                else:
                    raise ValueError(f"Número {n} está fora do intervalo (1-{len(opcoes_validas)}).")

            if not escolhas_finais:
                print("Você não selecionou nenhuma opção válida.")
                continue

            return list(set(escolhas_finais)) # Remove duplicatas

        except ValueError as e:
            print(f"Erro: {e}. Por favor, tente novamente (ex: 1, 3, 5).")

def coletar_dados_usuario(usuario: Usuario):
    """Executa o fluxo de perguntas de onboarding do app Nourish."""

    print("==========================================")
    [cite_start]print("       Bem-vindo ao Nourish! [cite: 13, 16]")
    print("==========================================")
    time.sleep(1)

    # [cite_start]1. Gênero [cite: 20]
    [cite_start]generos = ["Feminino", "Masculino", "Outros"] [cite: 21, 22, 23]
    usuario.genero = validar_input_lista("Qual seu gênero?", generos)

    # [cite_start]2. Altura (em metros) [cite: 26]
    usuario.altura_m = validar_input_float("Qual sua altura? (m)", 0.5, 3.0)

    # [cite_start]3. Peso (em kg) [cite: 30]
    usuario.peso_kg = validar_input_float("Qual seu peso atual? (kg)", 10.0, 300.0)

    # [cite_start]4. Prática de Exercício [cite: 34]
    [cite_start]exercicios = ["Faço", "Não faço"] [cite: 34]
    resposta_exercicio = validar_input_lista("Você faz exercício físico atualmente?", exercicios)
    usuario.faz_exercicio = (resposta_exercicio == "Faço")

    # [cite_start]5. Objetivo Principal [cite: 35]
    [cite_start]objetivos = ["Perder peso", "Ganhar massa", "Manter peso", "Reeducação alimentar"] [cite: 35]
    usuario.objetivo = validar_input_lista("Qual seu objetivo?", objetivos)

    # [cite_start]6. Motivação [cite: 36]
    [cite_start]motivacoes = ["Melhorar qualidade de vida", "Problema de saúde", "Controlar minha dieta", "Não desejo compartilhar"] [cite: 36]
    usuario.motivacao = validar_input_lista("Nos conte mais o por quê você quer chegar nesse objetivo:", motivacoes)

    # [cite_start]7. Preferências Alimentares [cite: 57]
    [cite_start]preferencias_opcoes = ["Carnes vermelhas", "Frango", "Massas", "Legumes", "Frutas", "Ovo"] [cite: 57]
    usuario.preferencias = validar_input_multiplo("O que você gosta de comer?", preferencias_opcoes)

    # Cálculo final do IMC e exibição do resumo
    usuario.calcular_imc()
    print(usuario)

    print("\n✅ Onboarding concluído com sucesso!")
    return usuario

# --- MÓDULO 3: Estrutura para Machine Learning (ML) ---

# O modelo de ML para recomendação exige que a base de dados de receitas seja estruturada
# para cálculo matricial (vetores de features).

def criar_dataframe_receitas(receitas_dict):
    """ Converte o dicionário de receitas em um DataFrame para análise de ML/Regras. """
    dados = []
    for categoria, lista_receitas in receitas_dict.items():
        for receita in lista_receitas:
            row = {
                'categoria': categoria,
                'nome': receita['nome'],
                'dificuldade': receita.get('dificuldade', 'N/A'),
                'tempo_preparo_min': receita.get('tempo_preparo_min', None),
                'calorias': receita['nutricional']['calorias'] if 'nutricional' in receita else None,
                'carboidratos': receita['nutricional']['carboidratos'] if 'nutricional' in receita else None,
                'proteinas': receita['nutricional']['proteinas'] if 'nutricional' in receita else None,
                'gordura': receita['nutricional']['gordura'] if 'nutricional' in receita else None,
                # Feature Engineering de Regras de Negócio
                'is_baixo_carb': 1 if receita['nome'] not in ['Frutas com mel', 'Nhoque de batata doce'] else 0, # Simulação de regra
                'is_alto_acucar': 1 if 'sorvete' in ''.join(receita['ingredientes']).lower() else 0
            }
            dados.append(row)
    return pd.DataFrame(dados)

def simular_recomendacao_regra(usuario: Usuario, df_receitas: pd.DataFrame):
    """ Função de Recomendação baseada em Regras de Negócio (MVP). """
    print("\n--- Simulação de Recomendação (MVP: Regras) ---")

    if usuario.objetivo == "Perder peso" and "Baixo Carboidrato" in usuario.preferencias:
        print(f"Filtro Ativo: {usuario.objetivo} + Baixo Carboidrato")

        # Filtrar receitas que não são alto açúcar e são consideradas baixo carboidrato
        receitas_filtradas = df_receitas[
            (df_receitas['is_alto_acucar'] == 0) &
            (df_receitas['is_baixo_carb'] == 1)
        ]

        if not receitas_filtradas.empty:
            sugestao = receitas_filtradas.sample(1)
            print(f"\n✅ Sugestão Otimizada: {sugestao['nome'].iloc[0]} ({sugestao['categoria'].iloc[0]})")
        else:
            # Note que as receitas do PDF são inadequadas para esta regra
            print("\n❌ Nenhuma receita da base de dados atual (PDF) atende ao critério 'Perda de Peso' + 'Baixo Carboidrato'.")
            print("Seria necessária uma receita como 'Omelete com Espinafre'.")

    else:
        # Recomendação padrão: receita mais simples/rápida
        print("\nSugestão Padrão: Nhoque de batata doce (Almoço/Jantar)")

    print("--------------------------------------------------")


# --- EXECUÇÃO DO NOTEBOOK ---

# 1. Coletar dados do usuário (simulação interativa)
usuario_novo = Usuario()
usuario_completo = coletar_dados_usuario(usuario_novo)

# 2. Estruturar o Dataset de Receitas
df_receitas = criar_dataframe_receitas(RECEITAS)
# print("\nEstrutura do Dataset de Receitas para ML:")
# display(df_receitas) # Use 'display' em ambientes Jupyter/Colab

# 3. Executar a simulação de recomendação
simular_recomendacao_regra(usuario_completo, df_receitas)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-1188202699.py, line 38)

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

# --- MÓDULO 4: Protótipo de Treinamento do Modelo de ML ---

# Reutilizando a Estrutura de Receitas e o Usuário Fictício:
# (Você precisará garantir que 'df_receitas' e 'usuario_completo' estejam definidos
# ou recriar um usuário fictício para a execução do notebook)

# Seção de Setup Rápido (para garantir a execução independente)
class UsuarioFicticio:
    genero = "Feminino"
    altura_m = 1.60
    peso_kg = 85.0
    imc = 33.20
    faz_exercicio = False
    objetivo = "Perder peso"
    motivacao = "Estética/Visual"
    preferencias = ["Baixo Carboidrato"]
    def calcular_imc(self): return self.imc
usuario_exemplo = UsuarioFicticio()

# DataFrame de Receitas (Simulação do Módulo 3)
dados_receitas = [
    {'nome': "Frutas com mel", 'categoria': "cafe_da_manha", 'calorias': 158, 'carboidratos': 25, 'proteinas': 59, 'gordura': 15, 'is_alto_acucar': 1, 'is_baixo_carb': 0},
    {'nome': "Nhoque de batata doce", 'categoria': "almoco_jantar", 'calorias': 350, 'carboidratos': 55, 'proteinas': 10, 'gordura': 8, 'is_alto_acucar': 0, 'is_baixo_carb': 0},
    {'nome': "Omelete de Legumes", 'categoria': "cafe_da_manha", 'calorias': 200, 'carboidratos': 5, 'proteinas': 25, 'gordura': 10, 'is_alto_acucar': 0, 'is_baixo_carb': 1}, # Receita ideal
    {'nome': "Salada de Quinoa", 'categoria': "almoco_jantar", 'calorias': 400, 'carboidratos': 40, 'proteinas': 18, 'gordura': 15, 'is_alto_acucar': 0, 'is_baixo_carb': 0},
    {'nome': "Bife com Batata Frita", 'categoria': "almoco_jantar", 'calorias': 650, 'carboidratos': 30, 'proteinas': 45, 'gordura': 40, 'is_alto_acucar': 0, 'is_baixo_carb': 0},
]
df_receitas = pd.DataFrame(dados_receitas)


def criar_dataset_treinamento(usuario: UsuarioFicticio, df_receitas: pd.DataFrame):
    """
    Cria um dataset combinando o perfil do usuário e as features da receita.
    Simula o Rótulo 'Y' (Interação Positiva) baseado em regras.
    """

    # 1. Expandir o Perfil do Usuário para todas as linhas de Receita
    df_usuario = pd.DataFrame([vars(usuario)]) # Cria um DF com 1 linha do usuário
    df_usuario = pd.concat([df_usuario] * len(df_receitas), ignore_index=True)

    # 2. Combinar Usuário e Receita
    df_treino = pd.concat([df_usuario, df_receitas], axis=1)

    # 3. Engenharia do Rótulo (Target Y): Simulação de Interação Positiva
    # Regra Simplificada de Negócio:
    # O usuário interage positivamente (Y=1) se:
    # a) O objetivo é "Perder peso" E a receita é "is_baixo_carb".
    # OU
    # b) A receita NÃO é alta em açúcar.

    df_treino['interacao_positiva'] = 0 # Default é 0 (Negativa)

    # Condição A: Usuário quer perder peso E a receita é Low Carb
    cond_peso_lowcarb = (df_treino['objetivo'] == 'Perder peso') & (df_treino['is_baixo_carb'] == 1)

    # Condição B: Se não é alto açúcar (independente do objetivo)
    cond_nao_alto_acucar = (df_treino['is_alto_acucar'] == 0)

    # Rótulo Final: Se A ou B (ou outras regras mais complexas)
    df_treino.loc[cond_peso_lowcarb | cond_nao_alto_acucar, 'interacao_positiva'] = 1

    # Ajuste manual para tornar o set mais desafiador:
    # A receita "Bife com Batata Frita" tem baixo açúcar mas muitas calorias (Não ideal para perda de peso)
    df_treino.loc[df_treino['nome'] == 'Bife com Batata Frita', 'interacao_positiva'] = 0

    return df_treino.drop(columns=['preferencias', 'motivacao', 'gordura', 'proteinas'])

# 4. Pré-processamento e Treinamento do Modelo (Regressão Logística)

def treinar_e_avaliar_modelo(df_treino: pd.DataFrame):
    """
    Executa o Pipeline de Pré-processamento, Treinamento e Avaliação.
    """

    # Definir Features (X) e Target (Y)
    X = df_treino.drop(columns=['interacao_positiva', 'nome', 'categoria'])
    Y = df_treino['interacao_positiva']

    # Divisão Treino/Teste (simulada devido ao tamanho pequeno do dataset)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)

    # Criar Coluna Transformer para Pré-processamento
    numeric_features = ['imc', 'altura_m', 'peso_kg', 'calorias', 'carboidratos']
    categorical_features = ['genero', 'objetivo']
    binary_features = ['faz_exercicio', 'is_baixo_carb', 'is_alto_acucar'] # Features que já são binárias (0/1)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features), # Padronização de valores numéricos (IMC, Calorias, etc.)
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # Criação de colunas binárias para variáveis categóricas
        ],
        remainder='passthrough' # Mantém as colunas binárias sem modificação
    )

    # Pipeline de Treinamento
    # 1. Pré-processar
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    # 2. Treinar o Modelo (Regressão Logística é um bom ponto de partida)
    model = LogisticRegression(solver='liblinear', random_state=42)
    model.fit(X_train_processed, Y_train)

    # 3. Previsão e Avaliação
    Y_pred = model.predict(X_test_processed)
    Y_proba = model.predict_proba(X_test_processed)[:, 1] # Probabilidade para métricas como AUC

    # Exibir Métricas de ML (Conforme a requisição)
    print("\n=======================================================")
    print("      📊 Avaliação das Métricas de Machine Learning")
    print("=======================================================")
    print(f"F1-Score (Equilíbrio Precisão/Recall): {f1_score(Y_test, Y_pred):.2f}")
    print(f"Precisão (Acerto das Sugestões Positivas): {precision_score(Y_test, Y_pred):.2f}")
    print(f"Recall (Cobertura das Boas Sugestões): {recall_score(Y_test, Y_pred):.2f}")

    try:
      print(f"AUC (Capacidade de Distinção): {roc_auc_score(Y_test, Y_proba):.2f}")
    except ValueError:
      print("AUC: Não calculável (apenas uma classe no conjunto de teste)")

    print("-------------------------------------------------------")

    return model, preprocessor


# --- EXECUÇÃO DO PROTÓTIPO DE ML ---

print("--- 1. Criação e Combinação do Dataset (Usuário + Receitas) ---")
df_prototipo = criar_dataset_treinamento(usuario_exemplo, df_receitas)
print(df_prototipo)

# ---------------------------------------------------------------------------------------------------------------------------------------------------
# Devido ao tamanho extremamente pequeno do dataset simulado (5 linhas), as métricas de ML terão valor limitado,
# mas a estrutura do pipeline de MLOps está correta.
# ---------------------------------------------------------------------------------------------------------------------------------------------------

# 2. Treinamento e Avaliação
modelo_final, preprocessor = treinar_e_avaliar_modelo(df_prototipo)

print("\n✅ Protótipo do Pipeline de ML (Feature Engineering, Pré-processamento e Treinamento) concluído!")
print("Este modelo agora serviria como base para o 'Serviço de Inferência' na arquitetura MLOps.")